In [1]:
import numpy as np
import pandas as pd

# Game Deck

In [2]:
class GameDeck():
    """ Create the game deck """
    values = ["A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K"]
    def __init__(self, num_decks=1, values=values, suits=4):
        self.num_decks = num_decks
        self.cards = values * num_decks * suits
        
    def __str__(self):
        return "{} cards left\n{}".format(len(self.cards), self.cards)
    
    def shuffle_cards(self):
        np.random.shuffle(self.cards)
        
    def deal_card(self, hand=None):
        if hand == None:
            return self.cards.pop(0)
        else:
            hand.append(self.cards.pop(0))

In [3]:
test = GameDeck()

In [4]:
print (test)
test.shuffle_cards()
print ()
print (test)
test.deal_card()
print()
print (test)

52 cards left
['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']

52 cards left
['Q', 9, 'Q', 3, 'K', 5, 'Q', 6, 4, 5, 7, 'K', 'A', 'J', 3, 'Q', 'K', 2, 5, 3, 'A', 'J', 9, 4, 7, 6, 8, 'J', 10, 'J', 4, 9, 'A', 5, 10, 'A', 7, 2, 2, 6, 10, 3, 10, 9, 2, 4, 8, 7, 8, 6, 'K', 8]

51 cards left
[9, 'Q', 3, 'K', 5, 'Q', 6, 4, 5, 7, 'K', 'A', 'J', 3, 'Q', 'K', 2, 5, 3, 'A', 'J', 9, 4, 7, 6, 8, 'J', 10, 'J', 4, 9, 'A', 5, 10, 'A', 7, 2, 2, 6, 10, 3, 10, 9, 2, 4, 8, 7, 8, 6, 'K', 8]


# Points calculator

In [5]:
def calc_ace(hand, count_A):
    # For each Ace, see if adding 11 causes it to bust
    # If no, then take Ace as 11
    # Else, take Ace as 1
    working_hand = hand.copy()
    
    # Remove Aces
    working_hand = [i if type(i) == int else 10 for i in working_hand if i != "A"]
    for k in range(count_A):
        if sum(working_hand) + 11 <= 21:
            working_hand.append(11)
        else:
            working_hand.append(1)
    return working_hand

In [6]:
def calc_points(hand):
    try:
        hand = hand.split(",")
    except:
        pass
    working_hand = hand.copy()
    
    if "A" in working_hand:
        
        # Count number of Aces
        count_A = working_hand.count("A")
        working_hand = calc_ace(working_hand, count_A)
        
    else:
        # Replace J, Q, and K with 10
        working_hand = [10 if i in ["J", "Q", "K"] else int(i) for i in working_hand]
    
    return sum(working_hand)

In [7]:
for i in [["A", "K"], [10,10], ["A", 5], ["A", "A", 8], [3,3]]:
    print (i, calc_points(i))

['A', 'K'] 21
[10, 10] 20
['A', 5] 16
['A', 'A', 8] 20
[3, 3] 6


# Soft 17

- Some casinos require the dealer to hit on a soft 17
- We need to know when it is a soft 17

In [8]:
def soft_17(hand):
    working_hand = hand.copy()
    Ace_11 = False
    if calc_points(working_hand) == 17 and "A" in working_hand:
        count_A = working_hand.count("A")
        working_hand = calc_ace(working_hand, count_A)
        if 11 in working_hand[-count_A:]:
            return True
    return False

In [9]:
for i in [["K", 7], ["A", 3, 3], ["A", "A", 5], ["A", "A", "A", 4]]:
    print (i, soft_17(i))

['K', 7] False
['A', 3, 3] True
['A', 'A', 5] True
['A', 'A', 'A', 4] True


# Play game

In [10]:
def start_game(num_decks=1):
    game_deck = GameDeck(num_decks=num_decks)
    
    d_hand = []
    p_hand = []
    
    game_deck.shuffle_cards()
    
    for _ in range(2):
        game_deck.deal_card(p_hand)
        game_deck.deal_card(d_hand)
        
    return game_deck, p_hand, d_hand

In [11]:
start_game()

(<__main__.GameDeck at 0x110f32198>, ['K', 9], [2, 'J'])

- strategies
    - 0: random
    - 1: recommended
    - 2: statistical analysis
    - 3: machine learning

In [12]:
def player_turn(game_deck, d_open, p_hand, strategy=0):
    new_hand = p_hand.copy()
    while calc_points(new_hand) <= 11:
        game_deck.deal_card(new_hand)
        
    if strategy == 0:
        while calc_points(new_hand) <= 18:
            if np.random.random() <= 0.5:
                game_deck.deal_card(new_hand)
            else:
                break
    elif strategy == 1:
        while calc_points(new_hand) <= 18:
            if d_open <= 6:
                game_deck.deal_card(new_hand)
            else:
                break
                
    return game_deck, p_hand, new_hand

In [13]:
test = start_game()
p_test = player_turn(test[0], test[1][0], test[2])
print (p_test)
print (calc_points(p_test[1]))
print (calc_points(p_test[2]))

(<__main__.GameDeck object at 0x10a22b8d0>, [3, 'K'], [3, 'K', 3, 'A', 10])
13
27


In [14]:
def dealer_turn(game_deck, hand, soft=True):
    new_hand = hand.copy()
    while calc_points(new_hand) < 17 or soft_17(new_hand):
        game_deck.deal_card(new_hand)
    
    return game_deck, hand, new_hand

In [15]:
test = start_game()
p_test = player_turn(test[0], test[1][0], test[2])
print (p_test)
print (calc_points(p_test[1]))
print (calc_points(p_test[2]))
d_test = dealer_turn(p_test[0], test[1])
print (d_test)
print (calc_points(d_test[1]))
print (calc_points(d_test[2]))

(<__main__.GameDeck object at 0x110f405c0>, [4, 9], [4, 9])
13
13
(<__main__.GameDeck object at 0x110f405c0>, ['K', 'A'], ['K', 'A'])
21
21


In [16]:
def l_to_s(l):
    l = [str(i) for i in l]
    return ",".join(l)

In [17]:
def play_split(game_deck, p_hand, d_hand, d_open, num_decks, strategy=0):
    to_split = np.random.random() <= 0.5
    if (strategy == 0 and to_split) or (strategy == 1 and p_hand[0] == 8):
        # Splits
        split = 1
        p_hand_1 = [p_hand[0]]
        p_hand_2 = [p_hand[1]]
        game_deck.deal_card(p_hand_1)
        game_deck.deal_card(p_hand_2)
        game_deck, p_hand_1, p_new_hand_1 = player_turn(game_deck, d_open, p_hand_1, strategy=strategy)
        game_deck, p_hand_2, p_new_hand_2 = player_turn(game_deck, d_open, p_hand_2, strategy=strategy)

        if calc_points(p_new_hand_1) > 21 and calc_points(p_new_hand_2) > 21:
            d_new_hand = d_hand.copy()
        else:
            game_deck, d_hand, d_new_hand = dealer_turn(game_deck, d_hand)

        return np.array([np.array([l_to_s(d_hand), l_to_s(d_new_hand), l_to_s(p_hand_1), l_to_s(p_new_hand_1), strategy, split, num_decks]), 
                np.array([l_to_s(d_hand), l_to_s(d_new_hand), l_to_s(p_hand_2), l_to_s(p_new_hand_2), strategy, split, num_decks])])
    else:
        return "No split"

In [18]:
def card_value(card):
    try:
        return int(card)
    except:
        if card == "A":
            return 1
        else:
            return 10

In [19]:
def play_game(num_decks=4, strategy=0):
    game_deck, p_hand, d_hand = start_game(num_decks=num_decks)
    
    d_open = card_value(d_hand[0])
    split = 0
    if calc_points(p_hand) != 21 and d_hand != 21:
        if (p_hand[0] == p_hand[1]):
            out = play_split(game_deck, p_hand, d_hand, d_open, num_decks, strategy=strategy)
        else:
            out = "No split"
        if type(out) == str:
            game_deck, p_hand, p_new_hand = player_turn(game_deck, d_open, p_hand, strategy=strategy)
            if calc_points(p_new_hand) > 21:
                d_new_hand = d_hand.copy()
            else:
                game_deck, d_hand, d_new_hand = dealer_turn(game_deck, d_hand)
            out = np.array([l_to_s(d_hand), l_to_s(d_new_hand), l_to_s(p_hand), l_to_s(p_new_hand), strategy, split, num_decks])
    else:
        p_new_hand = p_hand.copy()
        d_new_hand = d_hand.copy()
        out = np.array([l_to_s(d_hand), l_to_s(d_new_hand), l_to_s(p_hand), l_to_s(p_new_hand), strategy, split, num_decks])
    
    return out

In [20]:
play_game(strategy=1)

array(['4,4', '4,4,A', 'K,5', 'K,5,5', '1', '0', '4'], 
      dtype='<U5')

In [21]:
a = play_game()
while a.ndim < 2:
    a = play_game()
print (a)

[['K,3' 'K,3,J' '4,J' '4,J,7' '0' '1' '4']
 ['K,3' 'K,3,J' '4,2' '4,2,K' '0' '1' '4']]


In [22]:
a.ndim

2

# Generate cards

In [23]:
def gen_df(num_games=5000):
    data = []
    for _ in range(num_games):
        a = play_game(strategy=np.random.randint(2))
        if a.ndim == 2:
            data.extend(a)
        else:
            data.append(a)
            
    df = pd.DataFrame(data, columns=["d_hand", "d_final_hand", "p_hand", "p_final_hand", "strategy", "split", "num_decks"])
    return df

# Add columns

In [32]:
def add_columns(original):
    df = original.copy()
    df["strategy"] = df["strategy"].astype(int)
    df["split"] = df["split"].astype(int)
    df["num_decks"] = df["num_decks"].astype(int)
    
    df["d_open"] = df["d_hand"].apply(lambda x: card_value(x.split(",")[0]))
    df["d_initial"] = df["d_hand"].apply(calc_points)
    df["d_final"] = df["d_final_hand"].apply(calc_points)
    df["d_hit"] = df["d_final"]-df["d_initial"] > 0
    df["d_hit"] = df["d_hit"].astype(int)
    df["d_bust"] = df["d_final"] > 21
    df["d_bust"] = df["d_bust"].astype(int)
    
    df["p_initial"] = df["p_hand"].apply(calc_points)
    df["p_final"] = df["p_final_hand"].apply(calc_points)
    df["p_hit"] = df["p_final"]-df["p_initial"] > 0
    df["p_hit"] = df["p_hit"].astype(int)
    df["p_bust"] = df["p_final"] > 21
    df["p_bust"] = df["p_bust"].astype(int)
    
    df["p_win"] = (df["p_final"] > df["d_final"]) & (df["p_bust"] == 0) | (df["d_bust"] == 1)
    df["p_win"] = df["p_win"].astype(int)
    
    del df["d_hand"]
    del df["p_hand"]
    
    return df

# Generate CSVs

In [34]:
df = gen_df()

try:
    old_df = pd.read_csv("cards.csv")
    df = pd.concat([df, old_df])
except:
    pass
df.to_csv("cards.csv", index=False)

In [35]:
df = pd.read_csv("cards.csv")

In [36]:
df.shape

(66357, 7)

In [37]:
df2 = add_columns(df)

In [38]:
df2.to_csv("blackjack_data.csv", index=False)